In [ ]:
#Import Dependencies
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import os
import time
from bs4 import BeautifulSoup
import re
import pandas as pd
import requests

In [ ]:
#Configure Chromedriver

chrome_install = ChromeDriverManager().install()

folder = os.path.dirname(chrome_install)
chromedriver_path = os.path.join(folder, "chromedriver.exe")

In [ ]:
# Initialize Chrome WebDriver
browser = webdriver.Chrome(
    service = Service(chromedriver_path),
)

In [ ]:
#Setup search parameters
city = "toronto"
product = "Iphone 13"
min_price = 300
max_price = 600
days_listed = 1

In [ ]:
# Set up base URL
url = f'https://www.facebook.com/marketplace/{city}/search?query={product}&minPrice={min_price}&maxPrice={max_price}&daysSinceListed={days_listed}&exact=false'

# Visit the website
browser.get(url)

In [ ]:
# Locate the button with aria-label="Decline optional cookies" (Europe)
try:
    decline_button = browser.find_element(By.XPATH, '//div[@aria-label="Close" and @role="button"]')
    decline_button.click()
    print("Decline optional cookies button clicked!")
    
except:
    print("Could not find or click the optional cookies button!")
    pass

In [ ]:
# Locate the button for the login pop-up with aria-label="Close"
try:
    close_button = browser.find_element(By.XPATH, '//div[@aria-label="Close" and @role="button"]')
    close_button.click()
    print("Close button clicked!")
    
except:
    print("Could not find or click the close button!")
    pass

In [ ]:
#Scroll down to load all results
try:
    # Get the initial scroll position
    last_height = browser.execute_script("return document.body.scrollHeight")
    
    while True:
    
        # Scroll down to the bottom of the page using JavaScript
        browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(4)

        # Get the new scroll position
        new_height = browser.execute_script("return document.body.scrollHeight")

        # Check if we've reached the bottom
        if new_height == last_height:
            break
        

        # Update the scroll position
        last_height = new_height
        
        print("scrolled")
        
except Exception as e:
    print(f"An error occurred: {e}")

In [ ]:
# Retrieve the HTML
html = browser.page_source

# Use BeautifulSoup to parse the HTML
soup = BeautifulSoup(html, 'html.parser')

#Close the browser
browser.close()

In [ ]:
# Find all link elements
links = soup.find_all('a')

# Only keep items where the text matches your search terms and desired location
iphone_links = [link for link in links if product.lower() in link.text.lower() and city.lower() in link.text.lower()]

# Create empty list to store product data
iphone_data = []

# Store the items url and text into a list of dictionaries
for iphone_link in iphone_links:
    url = iphone_link.get('href')
    text = '\n'.join(iphone_link.stripped_strings)
    iphone_data.append({'text': text, 'url': url})

In [ ]:
# Create an empty list to store product data
extracted_data = []

for item in iphone_data:
    lines = item['text'].split('\n')

    # Regular expression to find numeric values
    numeric_pattern = re.compile('\d[\d,.]*')
    
    
    # Extracting prices
    # Iterate through lines to find the first line with numbers
    for line in lines:
        match = numeric_pattern.search(line)
        if match:    
            # Extract the first numeric value found
            price_str = match.group()
            # Convert price to float (handle commas)
            price = float(price_str.replace(',',''))
            break
            
    if price:
        print(f"Price extracted: {price}")
    else:
        print("price not found")

    # Extract title
    title = lines[-2]

    # Extract location
    location = lines[-1]

    # Add extracted data to a list of dictionaries
    extracted_data.append({
        'title': title,
        'price': price,
        'location': location,
        'url': re.sub(r'\?.*', '', item['url'])
        
    })

In [ ]:
# Convert extracted data into a Pandas Dataframe
items_df = pd.DataFrame(extracted_data)

# Sort the DataFrame by the "price" column in ascending order
sorted_df = items_df.sort_values(by='price')

# Get the 10 cheapest entries
cheapest_10 = sorted_df.head(10)

In [ ]:
# Create an empty message
message = ""

# Iterate over each row in the DataFrame containing the 10 cheapest items
for index, row in cheapest_10.iterrows():

    # Append the title, price, and URL of each item to the message string
    message += f"Title: {row['title']}\nPrice: {row['price']}\nURL: {row['url']}\n\n"


# URL of the Discord channel where the message will be posted
discord_url = 'PASTE REQUEST URL HERE'


# Payload containing the message to be sent
payload = {"content": message}


# Headers including the authorization token for the Discord API
headers = {"Authorization" : "PASTE AUTHORIZATION TOKEN HERE"}


# Send a POST request to the Discord API with the payload and headers
response = requests.post(discord_url, payload, headers = headers)
